# Plot bolus data

I still have a bit of work to do before getting to the data visualizations that live in my imagination (see the [part 3](https://michellejw.github.io/sugar/jupyter/2023/02/20/data_part03.html)). For example, I have not yet working on just basic plotting of insulin data (e.g., bolus, basal, corrections). I'm going to start with plotting bolus data here.